In [5]:
#nn.BatchNorm1d(120)
#nn.Dropout(p=0.5)
#max_pool2d(out, 2)
#.view(out.size(0), -1)
#nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
#nn.Tanh(),
from __future__ import print_function
import numpy as np
import torch  # import the torch library
import torch.nn as nn # use the nn module (class)
import torch.nn.functional as F    # use the nn module as function
import torch.optim as optim # optimization (i.e., SGD, ada,)
import torchvision # load the dataset
import torchvision.transforms as transforms # adjust the input image
import time # check the processing overhead

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    #transforms.Resize((64,64)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    #transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# total number of classes
#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the class for LeNet
class LeNet(nn.Module):
    def __init__(self):# will be called when you create an object
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.c1 = nn.Conv2d(3,16,11,stride=1, padding=5)
        self.b1 = nn.BatchNorm2d(16)
        self.c2 = nn.Conv2d(16,32,7,stride=1, padding=3)
        self.b2 = nn.BatchNorm2d(32)
        self.c3 = nn.Conv2d(32,64,5,1, padding=2)
        self.b3 = nn.BatchNorm2d(64)
        self.c4 = nn.Conv2d(64,64,5,1, padding=0)
        self.b4 = nn.BatchNorm2d(64)
        self.c5 = nn.Conv2d(64,64,5,1, padding=0)
        self.b5 = nn.BatchNorm2d(64)
        self.c6 = nn.Conv2d(64,320,5,1, padding=0)
        self.b6 = nn.BatchNorm2d(320)

        self.F1 = nn.Linear(320, 320)
        self.F2 = nn.Linear(320, 10)
        self.batchNorm1 = nn.BatchNorm1d(320)
        self.batchNorm2 = nn.BatchNorm1d(10)
        self.dropout = nn.Dropout(.5)
        self.MyRelu=nn.ReLU()
        self.activation = nn.ReLU()




    def forward(self, x):
        x=self.c1(x)
        x=self.b1(x)
        x=self.MyRelu(x)
        x=self.c2(x)
        x=self.b2(x)
        x=self.MyRelu(x)
        x=self.c3(x)
        x=self.b3(x)
        x=self.MyRelu(x)
        x=self.c4(x)
        x=self.b4(x)
        x=self.MyRelu(x)
        x=self.c5(nn.functional.max_pool2d(x,kernel_size=2,stride=2))
        x=self.b5(x)
        x=self.MyRelu(x)

        x=self.c6(nn.functional.max_pool2d(x,kernel_size=2,stride=2))
        x=self.b6(x)
        x=self.MyRelu(x)
        x=torch.flatten(x, 1)

        x=self.F1(x)
        x=self.batchNorm1(x)
        x=self.activation(x)
        x=self.dropout(x)
        x=self.F2(x)
        x=self.batchNorm2(x)
        x=self.activation(x)
        out = x




        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train() # set the model into training model (evaluation model in the testing)
    count = 0

    loss_Fn=nn.CrossEntropyLoss()

    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = data.to(device), target.to(device)

        out=model(data)
        loss=loss_Fn(out,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 32
    epochs = 50
    lr = 0.01
    no_cuda = False
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum = .9, weight_decay=1e-5)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time()
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))
if __name__ == '__main__':
    main()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.526412
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.997034
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.675990
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.535159

Test set: Average loss: -2.3401, Accuracy: 4816/10000 (48%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.558187
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.366179
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.357726
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.517596

Test set: Average loss: -3.0968, Accuracy: 5507/10000 (55%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.384630
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.158962
Train Epoch: 3 [25600/50000 (51%)]	Loss: 1.170381
Train Epoch: 3 [38400/50000 (77%)]	Loss: 1.292987

Test set: Average loss: -3.7316, Accuracy: 6201/10000 (62%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 1.355726
Train Epoch: 4 [12800/50000 (26%)]	Loss: 1.121731
Train Epoch: 4 [25600/50000 (51%)]	Lo